In [212]:
#Tunning_ANN

In [213]:
!pip install -U keras-tuner

In [214]:
#Importing New libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Dense
import keras_tuner as kf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [215]:
#Read The Dataset
df = pd.read_csv('https://raw.githubusercontent.com/KJanhavi2424/My-Datasets/main/Churn_Modelling.csv')

In [216]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [217]:
df.drop(columns = ['RowNumber' ,'CustomerId' ,'Surname'],inplace=True)

In [218]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [219]:
df = pd.get_dummies(data = df,columns=['Geography','Gender'])
df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,1,0,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,1,0,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,1,1,0,0,1,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0,1,0,0,1


In [220]:
x = df.drop(columns = ['Exited'])
y = df['Exited']

In [221]:
sc = StandardScaler()

x = sc.fit_transform(x)

In [222]:
from sklearn.model_selection import train_test_split

In [223]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size = 0.2,random_state=42)

In [224]:
ann = Sequential()

ann.add(Dense(units = 10,activation ='relu'))
ann.add(Dense(units = 1,activation ='sigmoid'))

ann.compile(optimizer = 'adadelta',loss = 'binary_crossentropy',metrics ='accuracy')

ann.fit(xtrain,ytrain,epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.6543 - accuracy: 0.6388
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.6538 - accuracy: 0.6396
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.6534 - accuracy: 0.6409
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.6529 - accuracy: 0.6421
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.6524 - accuracy: 0.6428
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.6519 - accuracy: 0.6441
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.6514 - accuracy: 0.6451
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.6509 - accuracy: 0.6463
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.6504 - accuracy: 0.6474
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.6499 - accura

In [225]:
ypred = ann.predict(xtest)
ypred = ypred > 0.5

63/63 [==============================] - 0s 959us/step


In [226]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.80      0.91      0.85      1607
           1       0.15      0.06      0.09       393

    accuracy                           0.74      2000
   macro avg       0.47      0.49      0.47      2000
weighted avg       0.67      0.74      0.70      2000



In [227]:
#Hyperparameter Tunning

In [228]:
def hyper(hp):
    model = Sequential()
    counter = 0
    for i in range(hp.Int('num_layers', min_value = 1,max_value=10)):
        if counter ==0:
            model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                            activation = hp.Choice('activation'+str(i),values = ['relu','tanh','sigmoid']),
                            input_dim = 13))
            counter = counter + 1
        else:
          model.add(Dense(
              hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
              activation = hp.Choice('activation'+str(i),values = ['relu','tanh','sigmoid'])))

    model.add(Dense(units = 1,activation='sigmoid'))
    model.compile(optimizer = hp.Choice('optimizer',values = ['adam','rmsprop']),
                loss = 'binary_crossentropy',metrics = 'accuracy')
    return model

In [229]:
tuner = kf.RandomSearch(hyper,
                        objective = 'val_accuracy',
                        max_trials = 3,
                        directory ='project',
                        project_name ='File1')

In [230]:
tuner.search(xtrain,ytrain,epochs = 5,validation_data = (xtest,ytest))

In [231]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3,
 'units0': 120,
 'activation0': 'tanh',
 'units1': 8,
 'activation1': 'relu',
 'optimizer': 'adam'}